# The Coin Change Problem Revisited

## Greedy Revisited

We know that we can solve the Coin Change Problem using a greedy algorithm.

In [143]:
def get_change_greedy(available_coins, change_to_give):
    available_coins.sort(reverse = True)
    change_given = []
    for c in available_coins:
        number = int(change_to_give / c)
        if number > 0:
            change_given = change_given + [c] * number
            change_to_give = change_to_give - c * number
    return change_given

In [144]:
get_change_greedy([1, 2, 5, 10, 50], 63)

[50, 10, 2, 1]

But we also know that we can easily find a sceanrio where greedy fails. In the following case the optimal solution would be to give 3 coins twice 10 cents and one 11 cent coin.

In [149]:
get_change_greedy([1, 2, 5, 10, 11, 50], 31)

[11, 11, 5, 2, 2]

## Complete Search

A possible solution to this problem is complete search. This strategy searches for all solutions and keeps only the smallest one.

In [146]:
def get_change_complete_search(available_coins, change_to_give):
    minimal_number_of_coins = change_to_give # worst case to give n one cent coins
    coins_for_change = []
    
    if change_to_give in available_coins:
        minimal_number_of_coins = 1
        coins_for_change = [change_to_give]
    else:
        global ping
        ping += 1

        for a_coin in [fitting_coin for fitting_coin in available_coins if fitting_coin < change_to_give]:
            coins_needed = [a_coin] + get_change_complete_search(available_coins, change_to_give - a_coin)
            coins_count = len(coins_needed)
            if (coins_count < minimal_number_of_coins):
                minimal_number_of_coins = coins_count
                coins_for_change = coins_needed
    return coins_for_change

In [148]:
ping = 0
get_change_complete_search([1, 2, 5, 10, 11, 50], 31)

[10, 10, 11]

There is one main problem with complete search. It really costs time. `get_change_complete_search(..)` tracks its number of calls where it has to try all the values by increasing a global variable `ping`. Lets see its value then.

In [150]:
ping

347944

Needless to say that an only little bit larger number shows the clear limits of this approach. Most probably you will  interrupt the kernel and calculate the change by hand if you don't have a coffee break to calm your customer during this time.

In [140]:
ping = 0
get_change_complete_search([1, 2, 5, 10, 21, 50], 63)

KeyboardInterrupt: 

In [141]:
ping

396354125

## Memoization Table

Since we learned that we run in a pretty large number of recursive calls which are calculating most results over and over again we could try an obvious approach to mitigate this problem: store already computed results in a table and return these if it comes to use such a result a second / third / etc. time.

So we introduce a table of lists (`known_results`) where we store a result as soon as we have it computed:

In [151]:
def get_change_memoization(available_coins, change_to_give, known_results):
    minimal_number_of_coins = change_to_give # worst case to give n one cent coins
    coins_for_change = []
    
    if change_to_give in available_coins:
        minimal_number_of_coins = 1
        coins_for_change = [change_to_give]
        known_results[change_to_give] = coins_for_change
        
    elif known_results[change_to_give]:
        coins_for_change = known_results[change_to_give]
    else:
        global ping
        ping += 1

        for a_coin in [fitting_coin for fitting_coin in available_coins if fitting_coin < change_to_give]:
            coins_needed = [a_coin] + get_change_memoization(available_coins, change_to_give - a_coin, known_results)
            coins_count = len(coins_needed)
            if (coins_count < minimal_number_of_coins):
                minimal_number_of_coins = coins_count
                coins_for_change = coins_needed
                known_results[change_to_give] = coins_for_change
    return coins_for_change

In [152]:
ping = 0
get_change_memoization([1, 2, 5, 10, 11, 50], 31, [[]]*32)

[10, 10, 11]

In [153]:
ping

26

OH. This brings hope back that we can use the computer to calculate the optimal change without getting a caffeine addiction. Lets try the impossible scenario from above:

In [154]:
ping = 0
get_change_memoization([1, 2, 5, 10, 21, 50], 63, [[]]* 64)

[21, 21, 21]

In [155]:
ping

57

## Dynamic Programming